In [1]:
import os
import cv2
import numpy as np
from tqdm import tqdm

In [2]:
REBUILD_DATA = False

class DogsVSCats():
    img_size = 100
    cats = "PetImages/Cat"
    dogs = "PetImages/Dog"

    labels = {cats: 0, dogs: 1}
    training_data = []

    cat_count = 0
    dog_count = 0

    def make_training_data(self):
        for label in self.labels:
            print(label)
            for f in tqdm(os.listdir(label)):
                    try:
                        path = os.path.join(label, f)
                        img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
                        img = cv2.resize(img, (self.img_size, self.img_size))
                        self.training_data.append([np.array(img), np.eye(2)[self.labels[label]]]) 

                        if label == self.cats:
                            self.cat_count += 1
                        elif label == self.dogs:
                            self.dog_count += 1

                    except Exception as e:
                        pass
                    

        np.random.shuffle(self.training_data)
        np.save("training_data.npy", np.array(self.training_data, dtype = object))
        print('Cats:',dogsvcats.cat_count)
        print('Dogs:',dogsvcats.dog_count)

if REBUILD_DATA:
    dogsvcats = DogsVSCats()
    dogsvcats.make_training_data()


In [3]:
training_data = np.load("training_data.npy", allow_pickle = True)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def convs(self, x):
        # max pooling over 2x2
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv3(x)), (2, 2))

        #print(x[0].shape)
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]*x[0].shape[2]
        return x
     def __init__(self):
        super().__init__() 
        self.conv1 = nn.Conv2d(1, 32, 5)
        self.conv2 = nn.Conv2d(32, 64, 5)
        self.conv3 = nn.Conv2d(64, 128, 5)

        x = torch.randn(50,50).view(-1,1,50,50)
        self._to_linear = None
        self.convs(x)

        self.fc1 = nn.Linear(self._to_linear, 512)
        self.fc2 = nn.Linear(512, 2)    

    def forward(self, x):
        x = self.convs(x)
        x = x.view(-1, self._to_linear)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim = 1)

net = Net()

In [5]:
import torch.optim as optim

optimizer = optim.Adam(net.parameters(), lr=0.001)
loss_function = nn.MSELoss()

X = torch.Tensor(np.array([i[0] for i in training_data])).view(-1,50,50)
X = X/255.0
y = torch.Tensor(np.array([i[1] for i in training_data]))

VAL_PCT = 0.2
val_size = int(len(X)*VAL_PCT)
print(val_size)

4989


In [6]:
train_X = X[:-val_size]
train_y = y[:-val_size]

test_X = X[-val_size:]
test_y = y[-val_size:]

print(len(train_X), len(test_X))

19957 4989


In [9]:
batch_size = 100
EPOCHS = 10

for epoch in range(EPOCHS):
    for i in tqdm(range(0, len(train_X), batch_size)): 
        #print(f"{i}:{i+batch_size}")
        batch_X = train_X[i:i+batch_size].view(-1, 1, 50, 50)
        batch_y = train_y[i:i+batch_size]

        net.zero_grad()
        outputs = net(batch_X)
        loss = loss_function(outputs, batch_y)
        loss.backward()
        optimizer.step()   

    print(loss)
    correct = 0
    total = 0
    with torch.no_grad():
        for i in tqdm(range(len(test_X))):
            real_class = torch.argmax(test_y[i])
            net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  
            predicted_class = torch.argmax(net_out)

            if predicted_class == real_class:
                correct += 1
            total += 1
    print("Accuracy: ", round(correct/total, 3))


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [01:17<00:00,  2.58it/s]


tensor(0.2299, grad_fn=<MseLossBackward0>)


100%|█████████████████████████████████████████████████████████████████████████████| 4989/4989 [00:18<00:00, 273.60it/s]


Accuracy:  0.638


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [01:14<00:00,  2.67it/s]


...

Accuracy:  0.753


100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [01:17<00:00,  2.57it/s]


tensor(0.0756, grad_fn=<MseLossBackward0>)


100%|█████████████████████████████████████████████████████████████████████████████| 4989/4989 [00:17<00:00, 278.68it/s]

Accuracy:  0.736


In [25]:
correct = 0
total = 0
with torch.no_grad():
    for i in tqdm(range(len(test_X))):
        real_class = torch.argmax(test_y[i])
        net_out = net(test_X[i].view(-1, 1, 50, 50))[0]  
        predicted_class = torch.argmax(net_out)

        if predicted_class == real_class:
            correct += 1
        total += 1
print("Accuracy: ", round(correct/total, 3))

100%|█████████████████████████████████████████████████████████████████████████████| 4989/4989 [00:18<00:00, 268.69it/s]

Accuracy:  0.754
